In [ ]:
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from datasets import load_dataset
import pandas as pd

pd.options.display.max_columns= None

In [ ]:
dataset = load_dataset("AnikaBasu/CyberbullyingDataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/2956 [00:00<?, ? examples/s]

In [ ]:
df = pd.DataFrame(dataset['train'])

In [ ]:
df.head()

,input,text,instruction,output
0,,Here is a classification that determines wheth...,"In other words #katandandre, your food was cra...",not_cyberbullying
1,,Here is a classification that determines wheth...,"In other words #katandandre, your food was cra...",not_cyberbullying
2,,Here is a classification that determines wheth...,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
3,,Here is a classification that determines wheth...,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
4,,Here is a classification that determines wheth...,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying


In [ ]:
def drop_columns(df, columns):
    return df.drop(columns=columns, inplace=False)

columns_to_drop = ['input', 'text']
df = drop_columns(df, columns_to_drop)

In [ ]:
df['output'].value_counts()

output
gender                 612
religion               562
other_cyberbullying    483
not_cyberbullying      434
age                    433
ethnicity              432
Name: count, dtype: int64

In [ ]:
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Assign values to cyberbullying types
cyberbullying_type_mapping = {
    "age": 1,
    "ethnicity": 2,
    "gender": 3,
    "not_cyberbullying": 4,
    "other_cyberbullying": 5,
    "religion": 6
}
df['output'] = df['output'].map(cyberbullying_type_mapping)

In [ ]:

def preprocess_text(text):

    # Lowercasing
    text = text.lower()

    # Replace links with 'link'
    text = re.sub(r'http\S+|www\S+', 'link', text)

    # Remove mentions (words starting with @)
    text = re.sub(r'@\w+', '', text)

    #remove hashtags
    text = re.sub(r'#\w+', '', text)

    #remove emoticons
    text = re.sub(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Removing Punctuation and Special Characters
    tokens = [word for word in tokens if word.isalnum()]

    # Removing Stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Join tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Applying text preprocessing to the 'instruction' column in the DataFrame
df['instruction'] = df['instruction'].apply(preprocess_text)

# Function to remove extra whitespaces from text
def remove_extra_whitespaces(text):
    return ' '.join(text.strip().split())

df['instruction'] = df['instruction'].apply(remove_extra_whitespaces)
print(df.head())

                                         instruction  output
0                            words food crapilicious       4
1                            words food crapilicious       4
2                                              white       4
3                   classy whore red velvet cupcakes       4
4  meh p thanks heads concerned another angry dud...       4


In [ ]:

#Tokenize data
from nltk.tokenize import word_tokenize

df['tokens'] = df['instruction'].apply(word_tokenize)

# Display the DataFrame with tokenized text
print(df[['instruction', 'tokens']].head())

                                         instruction  \
0                            words food crapilicious   
1                            words food crapilicious   
2                                              white   
3                   classy whore red velvet cupcakes   
4  meh p thanks heads concerned another angry dud...   

                                              tokens  
0                        [words, food, crapilicious]  
1                        [words, food, crapilicious]  
2                                            [white]  
3             [classy, whore, red, velvet, cupcakes]  
4  [meh, p, thanks, heads, concerned, another, an...  


In [ ]:
# NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# Function to remove stopwords
def remove_stopwords(text):
    return ' '.join(word for word in text.split() if word.lower() not in stop_words)

df['instruction'] = df['instruction'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df['instruction']

0                                 words food crapilicious
1                                 words food crapilicious
2                                                   white
3                        classy whore red velvet cupcakes
4       meh p thanks heads concerned another angry dud...
                              ...                        
2951    u tell ppl ok someone ridiculed ur race cuz so...
2952    right battle dumb fuck cuz whites see nigger e...
2953                    mad rt fuck obama dumb ass nigger
2954    fuck dumb nigger would never stand fuckin show...
2955    pussy ass bitch dumb muthafuker like fuck u ni...
Name: instruction, Length: 2956, dtype: object

In [ ]:
#Split in train, test and validation
from sklearn.model_selection import train_test_split

# Split data into train and test sets (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

#Making x_train and y_train columns
X_train = train_df['instruction']
y_train = train_df['output']

#Making y_test columns
X_test = test_df['instruction']
y_test = test_df['output']

In [ ]:
# Add new columns to main DataFrame for X_train, y_train, X_test, and y_test
df['X_train'] = X_train
df['y_train'] = y_train
df['X_test'] = X_test
df['y_test'] = y_test

# Save the modified DataFrame to a CSV file
df.to_csv('main_data_with_splits.csv', index=False)


In [ ]:
!pip install gensim

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import pandas as pd

In [ ]:
# Tokenize the sentences in the 'instruction' column
tokenized_instructions = [word_tokenize(sentence.lower()) for sentence in df['instruction']]

# Train Word2Vec model
model = Word2Vec(sentences=tokenized_instructions, vector_size=100, window=5, min_count=1, workers=4)

# Example: Get vector representation for the word 'document'
word = 'funny'
if word in model.wv:
    vector_representation = model.wv[word]
    print(f"Vector Representation for '{word}': {vector_representation}")
else:
    print(f"Word '{word}' is not present in the vocabulary.")

Vector Representation for 'funny': [-0.03751611  0.06452203 -0.02186869  0.00585577  0.05335483 -0.21411765
  0.04922131  0.2578973  -0.08609331 -0.06855959 -0.06158377 -0.18535167
 -0.04983553  0.06307922  0.03210637 -0.07114465 -0.00731555 -0.08967033
  0.02096597 -0.25634673  0.08709929  0.08608868  0.06220416 -0.07909789
  0.02961295 -0.01879383 -0.07316502 -0.07874826 -0.1336306  -0.00218034
  0.16122015  0.05447409  0.05388979 -0.12111138 -0.04750177  0.13774405
  0.01788874 -0.10989689 -0.05186755 -0.2506227   0.00872856 -0.13900103
 -0.05736784 -0.02622901  0.14907835  0.00132204 -0.10551232 -0.03239736
  0.05475273  0.09145422  0.08250487 -0.1316005  -0.07896489 -0.02421799
 -0.08260739  0.06598065 -0.01502464 -0.0277908  -0.1303557   0.02447626
  0.09821928  0.06734305 -0.04075884  0.00588388 -0.11809616  0.15566872
  0.02177693  0.06047384 -0.13622284  0.18504404 -0.11642493  0.08963665
  0.15351424 -0.05166678  0.17727341  0.06077969  0.0134224  -0.01922006
 -0.04898158  0.

In [ ]:
# Tokenize the documents (test data)
tokenized_documents = [word_tokenize(sentence.lower()) for sentence in test_df['instruction']]

# Train Word2Vec model
model = Word2Vec(sentences=tokenized_documents, vector_size=100, window=5, min_count=1, workers=4)

# Example: Get vector representation for the word 'document'
vector_representation = model.wv['funny']
print("Vector Representation for 'document':", vector_representation)

Vector Representation for 'document': [ 5.5988291e-03 -5.1964680e-03  8.0693308e-03  6.6305045e-03
 -7.1736057e-03 -9.3747312e-03 -6.2638875e-03 -5.8589205e-03
  7.2525167e-03  3.2825251e-03 -9.6451994e-03 -6.9406861e-04
 -9.6254284e-03 -4.2355978e-03 -3.4052413e-03  9.4269559e-04
  4.8786765e-03 -6.5500853e-03  1.1262280e-03 -5.0327796e-03
  2.5653387e-03  2.2930715e-03 -1.4162185e-03  7.3727448e-03
  6.2394231e-03  4.1047228e-03  4.8389887e-03  2.6239892e-03
 -2.9911424e-03  5.0245770e-03 -1.0800774e-03 -2.3107708e-03
  3.6596865e-03  9.1353366e-03 -5.0258526e-04  2.8666120e-03
 -9.2095984e-03 -5.3261491e-03 -4.1585234e-03 -1.0268232e-02
  9.9928817e-03 -8.2253357e-03 -7.3876730e-03 -6.1279340e-03
  3.1728437e-03  4.7464953e-03 -4.3718121e-03  6.2713488e-03
 -7.9288511e-05  9.0606278e-03 -2.8761432e-03  2.2330473e-03
  5.9018251e-03 -4.2238641e-03 -3.1818827e-03  7.1901623e-03
 -2.4823246e-03  8.9776991e-03 -2.3066974e-03  4.0544672e-03
 -4.2728200e-03 -8.4045958e-03  3.4808961e-04 -

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd


In [ ]:
# Tokenize the documents
tokenized_train_documents = [word_tokenize(sentence.lower()) for sentence in X_train]
tokenized_test_documents = [word_tokenize(sentence.lower()) for sentence in X_test]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_train_documents, vector_size=100, window=5, min_count=1, workers=4)

# Get Word2Vec embeddings for each document in train and test sets
X_train_word2vec = [[word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv.key_to_index] for tokens in tokenized_train_documents]
X_test_word2vec = [[word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv.key_to_index] for tokens in tokenized_test_documents]

# Average Word2Vec embeddings for each document
X_train_word2vec = [sum(embeddings) / len(embeddings) if embeddings else [0] * 100 for embeddings in X_train_word2vec]
X_test_word2vec = [sum(embeddings) / len(embeddings) if embeddings else [0] * 100 for embeddings in X_test_word2vec]

In [ ]:
# Logistic Regression
logistic_regression = LogisticRegression(max_iter=1000)
logistic_regression.fit(X_train_word2vec, y_train)
logistic_regression_predictions_train = logistic_regression.predict(X_train_word2vec)

# Evaluation
logistic_regression_accuracy_train = accuracy_score(y_train, logistic_regression_predictions_train)
logistic_regression_precision_train = precision_score(y_train, logistic_regression_predictions_train, average='weighted')
logistic_regression_recall_train = recall_score(y_train, logistic_regression_predictions_train, average='weighted')
logistic_regression_f1_train = f1_score(y_train, logistic_regression_predictions_train, average='weighted')

print("Logistic Regression Scores (Training Set):")
print("Accuracy:", logistic_regression_accuracy_train)
print("Precision:", logistic_regression_precision_train)
print("Recall:", logistic_regression_recall_train)
print("F1-Score:", logistic_regression_f1_train)
print()

Logistic Regression Scores (Training Set):
Accuracy: 0.3460236886632826
Precision: 0.4164422897194835
Recall: 0.3460236886632826
F1-Score: 0.337519172883348



In [ ]:
# Logistic Regression
logistic_regression = LogisticRegression(max_iter=1000)
logistic_regression.fit(X_test_word2vec, y_test)
logistic_regression_predictions_test = logistic_regression.predict(X_test_word2vec)

# Evaluation
logistic_regression_accuracy_test = accuracy_score(y_test, logistic_regression_predictions_test)
logistic_regression_precision_test = precision_score(y_test, logistic_regression_predictions_test, average='weighted')
logistic_regression_recall_test = recall_score(y_test, logistic_regression_predictions_test, average='weighted')
logistic_regression_f1_test = f1_score(y_test, logistic_regression_predictions_test, average='weighted')

print("Logistic Regression Scores (Test Set):")
print("Accuracy:", logistic_regression_accuracy_test)
print("Precision:", logistic_regression_precision_test)
print("Recall:", logistic_regression_recall_test)
print("F1-Score:", logistic_regression_f1_test)
print()

Logistic Regression Scores (Test Set):
Accuracy: 0.3125
Precision: 0.3894417414342569
Recall: 0.3125
F1-Score: 0.25287389791303005



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Support Vector Machine (SVM)
svm = SVC()
svm.fit(X_train_word2vec, y_train)
svm_predictions_train = svm.predict(X_train_word2vec)

# Evaluation
svm_accuracy_train = accuracy_score(y_train, svm_predictions_train)
svm_precision_train = precision_score(y_train, svm_predictions_train, average='weighted')
svm_recall_train = recall_score(y_train, svm_predictions_train, average='weighted')
svm_f1_train = f1_score(y_train, svm_predictions_train, average='weighted')

print("SVM Scores (Training Set):")
print("Accuracy:", svm_accuracy_train)
print("Precision:", svm_precision_train)
print("Recall:", svm_recall_train)
print("F1-Score:", svm_f1_train)
print()

# Support Vector Machine (SVM)
svm = SVC()
svm.fit(X_test_word2vec, y_test)
svm_predictions_test = svm.predict(X_test_word2vec)

# Evaluation
svm_accuracy_test = accuracy_score(y_test, svm_predictions_test)
svm_precision_test = precision_score(y_test, svm_predictions_test, average='weighted')
svm_recall_test = recall_score(y_test, svm_predictions_test, average='weighted')
svm_f1_test = f1_score(y_test, svm_predictions_test, average='weighted')

print("SVM Scores (Test Set):")
print("Accuracy:", svm_accuracy_test)
print("Precision:", svm_precision_test)
print("Recall:", svm_recall_test)
print("F1-Score:", svm_f1_test)
print()

SVM Scores (Training Set):
Accuracy: 0.4750423011844332
Precision: 0.5615544007269011
Recall: 0.4750423011844332
F1-Score: 0.4851461304573575

SVM Scores (Test Set):
Accuracy: 0.38344594594594594
Precision: 0.4232841653702801
Recall: 0.38344594594594594
F1-Score: 0.34894155570692126



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Neural Network
nn_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
nn_model.fit(X_train_word2vec, y_train)
nn_predictions_train = nn_model.predict(X_train_word2vec)

# Evaluation
nn_accuracy_train = accuracy_score(y_train, nn_predictions_train)
nn_precision_train = precision_score(y_train, nn_predictions_train, average='weighted')
nn_recall_train = recall_score(y_train, nn_predictions_train, average='weighted')
nn_f1_train = f1_score(y_train, nn_predictions_train, average='weighted')

print("Neural Network Scores (Training Set):")
print("Accuracy:", nn_accuracy_train)
print("Precision:", nn_precision_train)
print("Recall:", nn_recall_train)
print("F1-Score:", nn_f1_train)
print()

# Neural Network
nn_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
nn_model.fit(X_test_word2vec, y_test)
nn_predictions_test = nn_model.predict(X_test_word2vec)

# Evaluation
nn_accuracy_test = accuracy_score(y_test, nn_predictions_test)
nn_precision_test = precision_score(y_test, nn_predictions_test, average='weighted')
nn_recall_test = recall_score(y_test, nn_predictions_test, average='weighted')
nn_f1_test = f1_score(y_test, nn_predictions_test, average='weighted')

print("Neural Network Scores (Test Set):")
print("Accuracy:", nn_accuracy_test)
print("Precision:", nn_precision_test)
print("Recall:", nn_recall_test)
print("F1-Score:", nn_f1_test)
print()

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Neural Network Scores (Training Set):
Accuracy: 0.6742808798646363
Precision: 0.6669324944805682
Recall: 0.6742808798646363
F1-Score: 0.6656012468336602

Neural Network Scores (Test Set):
Accuracy: 0.6706081081081081
Precision: 0.68198539483454
Recall: 0.6706081081081081
F1-Score: 0.6605537198046229



/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
# Tokenization and Padding for LSTM
maxlen = 100  # Maximum sequence length
max_words = 10000  # Maximum number of words to consider as features

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_lstm = tokenizer.texts_to_sequences(X_train)
X_train_lstm = pad_sequences(X_train_lstm, maxlen=maxlen)

# Define LSTM model architecture
lstm_model = Sequential()
lstm_model.add(Embedding(max_words, 32, input_length=maxlen))
lstm_model.add(LSTM(32))
lstm_model.add(Dense(1, activation='sigmoid'))

# Compile and train LSTM model
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=32)

# Make predictions on train set
lstm_train_preds = (lstm_model.predict(X_train_lstm) > 0.5).astype(int)

# Evaluation
lstm_accuracy_train = accuracy_score(y_train, lstm_train_preds.flatten())
lstm_precision_train = precision_score(y_train, lstm_train_preds.flatten(), average='weighted')
lstm_recall_train = recall_score(y_train, lstm_train_preds.flatten(), average='weighted')
lstm_f1_train = f1_score(y_train, lstm_train_preds.flatten(), average='weighted')

print("LSTM Scores (Training Set):")
print("Accuracy:", lstm_accuracy_train)
print("Precision:", lstm_precision_train)
print("Recall:", lstm_recall_train)
print("F1-Score:", lstm_f1_train)
print()



Epoch 1/10
74/74 [==============================] - 18s 98ms/step - loss: -10.6393 - accuracy: 0.1459
Epoch 2/10
74/74 [==============================] - 6s 84ms/step - loss: -27.6945 - accuracy: 0.1485
Epoch 3/10
74/74 [==============================] - 3s 38ms/step - loss: -36.2965 - accuracy: 0.1485
Epoch 4/10
74/74 [==============================] - 3s 38ms/step - loss: -43.5698 - accuracy: 0.1485
Epoch 5/10
74/74 [==============================] - 3s 38ms/step - loss: -50.4423 - accuracy: 0.1485
Epoch 6/10
74/74 [==============================] - 4s 52ms/step - loss: -57.1059 - accuracy: 0.1485
Epoch 7/10
74/74 [==============================] - 3s 37ms/step - loss: -63.6385 - accuracy: 0.1485
Epoch 8/10
74/74 [==============================] - 3s 38ms/step - loss: -70.1094 - accuracy: 0.1485
Epoch 9/10
74/74 [==============================] - 3s 38ms/step - loss: -76.5155 - accuracy: 0.1485
Epoch 10/10
74/74 [==============================] - 1s 11ms/step
LSTM Scores (Training Se

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Tokenization and Padding for LSTM
maxlen = 100  # Maximum sequence length
max_words = 10000  # Maximum number of words to consider as features

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_test)

X_test_lstm = tokenizer.texts_to_sequences(X_test)
X_test_lstm = pad_sequences(X_test_lstm, maxlen=maxlen)

# Define LSTM model architecture
lstm_model = Sequential()
lstm_model.add(Embedding(max_words, 32, input_length=maxlen))
lstm_model.add(LSTM(32))
lstm_model.add(Dense(1, activation='sigmoid'))

# Compile and train LSTM model
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_test_lstm, y_test, epochs=10, batch_size=32)

# Make predictions on train set
lstm_test_preds = (lstm_model.predict(X_test_lstm) > 0.5).astype(int)

# Evaluation
lstm_accuracy_test = accuracy_score(y_test, lstm_test_preds.flatten())
lstm_precision_test = precision_score(y_test, lstm_test_preds.flatten(), average='weighted')
lstm_recall_test = recall_score(y_test, lstm_test_preds.flatten(), average='weighted')
lstm_f1_test = f1_score(y_test, lstm_test_preds.flatten(), average='weighted')

print("LSTM Scores (Test Set):")
print("Accuracy:", lstm_accuracy_test)
print("Precision:", lstm_precision_test)
print("Recall:", lstm_recall_test)
print("F1-Score:", lstm_f1_test)
print()

Epoch 1/10
19/19 [==============================] - 8s 83ms/step - loss: 0.2473 - accuracy: 0.1368
Epoch 2/10
19/19 [==============================] - 2s 87ms/step - loss: -5.9014 - accuracy: 0.1385
Epoch 3/10
19/19 [==============================] - 1s 38ms/step - loss: -13.2138 - accuracy: 0.1385
Epoch 4/10
19/19 [==============================] - 1s 38ms/step - loss: -17.8136 - accuracy: 0.1385
Epoch 5/10
19/19 [==============================] - 1s 44ms/step - loss: -21.6370 - accuracy: 0.1385
Epoch 6/10
19/19 [==============================] - 1s 63ms/step - loss: -24.6750 - accuracy: 0.1385
Epoch 7/10
19/19 [==============================] - 1s 64ms/step - loss: -27.2452 - accuracy: 0.1385
Epoch 8/10
19/19 [==============================] - 1s 38ms/step - loss: -29.5790 - accuracy: 0.1385
Epoch 9/10
19/19 [==============================] - 1s 39ms/step - loss: -31.7282 - accuracy: 0.1385
Epoch 10/10
19/19 [==============================] - 1s 11ms/step
LSTM Scores (Test Set):
Accu

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
